In [ ]:
import config

import pandas as pd
import numpy as np
import joblib
from collections import Counter

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


In [ ]:
# Loading in the mRNA and clinical data:
clinical_df = pd.read_csv("ucec_tcga_pan_can_atlas_2018\data_clinical_patient.txt", sep="\t", comment="#", low_memory=False)
clinical_df = clinical_df.set_index('PATIENT_ID')

mrna_df = pd.read_csv("ucec_tcga_pan_can_atlas_2018/data_mrna_seq_v2_rsem_zscores_ref_all_samples.txt", sep="\t", comment="#")

# There are 527 patients in the mRNA and 529 patients in the clinical data

# The first 2 columns of the mRNA data are labels (Hugo_Symbol then Entrez_Gene_Id). 
# 13 of the genes do not have Hugo_symbols, so for these I will you the Entrex_Gene_Id as the label.
missing_symbols = mrna_df['Hugo_Symbol'].isnull()
mrna_df.loc[missing_symbols, 'Hugo_Symbol'] = mrna_df.loc[missing_symbols, 'Entrez_Gene_Id'].astype(str)

# There are 7 rows that have both the same Hugo_Symbol and Entrez_Gene_Id but different values for the patients.
# I will rename these rows to have unique labels by appending -1-of-2 and -2-of-2 to the Hugo_Symbol.
# Get value counts
counts = mrna_df['Hugo_Symbol'].value_counts()

# Generate unique labels for duplicates
def label_duplicates(value, index):
    if counts[value] == 1:
        return value  # Keep unique values unchanged
    occurrence = mrna_df.groupby('Hugo_Symbol').cumcount() + 1  # Count occurrences per group
    return f"{value}-{occurrence[index]}-of-{counts[value]}"

# Apply the labeling function
mrna_df['Hugo_Symbol'] = [label_duplicates(value, idx) for idx, value in mrna_df['Hugo_Symbol'].items()]

mrna_df = mrna_df.set_index('Hugo_Symbol')
mrna_df = mrna_df.drop(columns="Entrez_Gene_Id") # removing the label column before I transpose the df
mrna_df= mrna_df.transpose() # now the patients are the index and the genes are the columns
mrna_df.index = [id[:-3] for id in mrna_df.index] # removes extranious -01 so that the patient ids match the clinical data



In [ ]:
def assign_label(row):
    '''given a row assigns 1 for recurrance, 0 for no recurrance, 
    and None if the patient has no recurrence information. 
    Currently uses NEW_TUMOR_EVENT_AFTER_INITIAL_TREATMENT to identify recurrance.
    If NEW_TUMOR_EVENT_AFTER_INITIAL_TREATMENT is NaN, uses DSF_STATUS to save the label.'''
    if row['NEW_TUMOR_EVENT_AFTER_INITIAL_TREATMENT'] == 'Yes':
        return 1
    elif row['NEW_TUMOR_EVENT_AFTER_INITIAL_TREATMENT'] == 'No':
        return 0
    elif pd.isna(row['NEW_TUMOR_EVENT_AFTER_INITIAL_TREATMENT']):
        if row['DFS_STATUS'] == '1:Recurred/Progressed':
            return 1
        elif row['DFS_STATUS'] == '0:DiseaseFree':
            return 0
        else:
            return None


def drop_patients_missing_data_and_get_labels(clnical_df, mrna_df):
    '''Drops patients from both dataframes that are not present in the other dataframe. 
    Drops patients who are missing labeling data used to define recurrence.
    Returns the cleaned dataframes and labels.'''
    # Find patient IDs not shared between the two dataframes:
    clinical_not_in_mrna = set(clinical_df.index) - set(mrna_df.index)
    mrna_not_in_clinical = set(mrna_df.index) - set(clinical_df.index)
    # There are 2 patients ('TCGA-EY-A1GJ', 'TCGA-AP-A0LQ') in the clinical data that are not in the mRNA data.
    clinical_df = clinical_df.drop(index=clinical_not_in_mrna)
    mrna_df = mrna_df.drop(index=mrna_not_in_clinical)

    # Now drop patients missing labeling data used to define recurrence:
    labels = clinical_df.index.to_series().map(assign_label)

    patients_no_label = labels[labels.isna()].index
    clinical_df = clinical_df.drop(index=patients_no_label)
    mrna_df = mrna_df.drop(index=patients_no_label)
    labels = labels.drop(index=patients_no_label)
    assert not labels.isna().any(), "Found unlabeled patient after cleaning"

    return clinical_df, mrna_df, labels

In [ ]:
def drop_post_diagnosis_clinical_columns(clinical_df):
    '''Removes all columns in the clinical data that are recurrence indicators or are not available at diagnosis.
    Returns the cleaned clinical dataframe and the labels series.'''
    cols_to_drop = [
    "DAYS_LAST_FOLLOWUP",              # follow-up time after diagnosis (future info)
    "FORM_COMPLETION_DATE",            # administrative metadata, not predictive
    "INFORMED_CONSENT_VERIFIED",       # administrative, no biological meaning
    "NEW_TUMOR_EVENT_AFTER_INITIAL_TREATMENT",  # recurrence event → direct leakage
    "PERSON_NEOPLASM_CANCER_STATUS",   # disease status at follow-up → leakage
    "IN_PANCANPATHWAYS_FREEZE",        # technical/analysis flag, not biological
    "OS_STATUS",                       # overall survival outcome → leakage
    "OS_MONTHS",                       # overall survival time → leakage
    "DSS_STATUS",                      # disease-specific survival outcome → leakage
    "DSS_MONTHS",                      # disease-specific survival time → leakage
    "DFS_STATUS",                      # disease-free survival outcome → leakage
    "DFS_MONTHS",                      # disease-free survival time → leakage
    "PFS_STATUS",                      # progression-free survival outcome → leakage
    "PFS_MONTHS"                       # progression-free survival time → leakage
]
    clinical_df = clinical_df.drop(columns=cols_to_drop)
    return clinical_df  

In [ ]:
from sklearn.model_selection import train_test_split

def split_train_test(clinical_df, mrna_df, labels, test_size=0.2, random_state=42):
    """
    Splits clinical and mRNA data into train/test sets using precomputed labels.

    Parameters
    ----------
    clinical_df : pd.DataFrame
        Clinical features (indexed by patient ID).
    mrna_df : pd.DataFrame
        mRNA expression features (indexed by patient ID).
    labels : pd.Series
        Precomputed labels indexed by patient ID.
    test_size : float
        Fraction of patients to hold out for testing.
    random_state : int
        Random seed for reproducibility.

    Returns
    -------
    dict of train/test splits:
        {
            "X_clinical_train", "X_clinical_test",
            "X_mrna_train", "X_mrna_test",
            "y_train", "y_test"
        }
    """

    # Train/test split on patient IDs
    train_ids, test_ids = train_test_split(
        labels.index,
        test_size=test_size,
        stratify=labels,
        random_state=random_state
    )

    # Slice dataframes and labels
    splits = {
        "X_clinical_train": clinical_df.loc[train_ids],
        "X_clinical_test":  clinical_df.loc[test_ids],
        "X_mrna_train":     mrna_df.loc[train_ids],
        "X_mrna_test":      mrna_df.loc[test_ids],
        "y_train":          labels.loc[train_ids],
        "y_test":           labels.loc[test_ids]
    }

    return splits

In [ ]:
# Remove any columns in the clinical data that are recurrence indicators or are not available at diagnosis.

In [ ]:
clinical_df, mrna_df, labels = drop_patients_missing_data_and_get_labels(clinical_df, mrna_df)
clinical_df = drop_post_diagnosis_clinical_columns(clinical_df)
splits = split_train_test(clinical_df, mrna_df, labels)

X_clinical_train = splits["X_clinical_train"]
X_clinical_test  = splits["X_clinical_test"]
X_mrna_train     = splits["X_mrna_train"]
X_mrna_test      = splits["X_mrna_test"]
y_train          = splits["y_train"]
y_test           = splits["y_test"]

